# Changing the Leetcode stats in my CV

## 1. get data from Leetcode

In [1]:
import requests
import json

from config import USERNAME


def get_leetcode_profile_data(username):
    # using an api for getting Leetcode stats
    url = f"https://leetcode-stats-api.herokuapp.com/{username}/"
    response = requests.get(url)

    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
        # Decode the byte content to a string
        response_content = response.content.decode('utf-8')

        # Parse the JSON string into a dictionary
        profile_data = json.loads(response_content)

        return profile_data
    else:
        print("Failed to retrieve data.")
        return None


profile_data = get_leetcode_profile_data(USERNAME)

# total = profile_data["totalSolved"]
easy = profile_data["easySolved"]
medium = profile_data["mediumSolved"]
hard = profile_data["hardSolved"]
# rank = profile_data["ranking"]

## 2. Copy the template

In [2]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

from config import TEMPLATE_ID, CREDENTIALS_DICT

SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# SERVICE_ACCOUNT_FILE = 'credentials.json'
credentials = service_account.Credentials.from_service_account_info(CREDENTIALS_DICT)
credentials = credentials.with_scopes(SCOPES)

# Initialize the Google Drive service
drive_service = build('drive', 'v3', credentials=credentials)

# Initialize the Google Docs service
docs_service = build('docs', 'v1', credentials=credentials)

template_id = TEMPLATE_ID

copy_title = 'tmp'
body = {
    'name': copy_title
}

drive_response = drive_service.files().copy(
    fileId=template_id, body=body).execute()
document_copy_id = drive_response.get('id')

## 3. Replace placeholders

In [3]:
import datetime

replacements = {
    '{easy_solved}': str(easy),
    '{medium_solved}': str(medium),
    '{hard_solved}': str(hard)
}

# Create a list of requests to replace each placeholder
requests = []
for placeholder, value in replacements.items():
    requests.append({
        'replaceAllText': {
            'containsText': {
                'text': placeholder,
                'matchCase': True
            },
            'replaceText': value,
        }
    })

requests.append({
    'replaceAllText': {
        'containsText': {
            'text': '{date}',
            'matchCase': True
        },
        'replaceText': datetime.datetime.now().strftime("%Y-%m-%d"),
    }
})

result = docs_service.documents().batchUpdate(
    documentId=document_copy_id, body={'requests': requests}).execute()


## 4. Delete duplicate CV.pdf

In [4]:
from config import FOLDER_ID

folder_id = FOLDER_ID
file_name = 'CV.pdf'

query = f'"{folder_id}" in parents and trashed=false and name="{file_name}"'
results = drive_service.files().list(q=query).execute()
files = results.get('files', [])
if not files:
    print("No file found.")
else:
    file_id = files[0]['id']
    drive_service.files().delete(fileId=file_id).execute()

## 5. Export to PDF

In [5]:
pdf_file = drive_service.files().export(fileId=document_copy_id, mimeType='application/pdf').execute()

## 6. Upload the PDF

In [6]:
from googleapiclient.http import MediaIoBaseUpload
import io

from config import FOLDER_ID

# upload it
file_metadata = {
    'name': 'CV.pdf',
    'parents': [FOLDER_ID]
}

pdf_file_io = io.BytesIO(pdf_file)
media = MediaIoBaseUpload(pdf_file_io, mimetype='application/pdf')

uploaded_file = drive_service.files().create(
    body=file_metadata,
    media_body=media,
    fields='id'
).execute()

## 7. Delete the tmp file

In [7]:
drive_service.files().delete(fileId=document_copy_id).execute()

''